#### 对于场次洪水数据，由于每个场次的洪水我都设置了一个月的预热期，所以将预报的数据还原为场次洪水数据只需要碰到实测入流为空+实测入流有数的时候就为一场场次洪水，按时间命名为1.csv\2.csv.....。对于松辽委预报的场次数据，我们按时间排列为1.csv\2.csv....，这就将我们使用新安江模型预报的数据和松辽委预报的数据对应起来了。（如此处理的原因是在模型的输入的时候，时间不能不连续，所以场次洪水的时间要重新设置，没办法直接按时间对应还原场次洪水）

In [1]:
#注，要全部都是英文

import netCDF4 as nc
import pandas as pd
import numpy as np

file_path = r"C:\Users\20676\Desktop\songliaoyubao\dahuofang\dahuofangchangcishuju\xaj_evaluation_results (2).nc"
data = nc.Dataset(file_path)

# 读取变量
time = nc.num2date(data.variables['time'][:], data.variables['time'].units)
qsim = data.variables['qsim'][:]
qobs = data.variables['qobs'][:]
prcp = data.variables['prcp'][:]

# 将变量转换为pandas DataFrame
df = pd.DataFrame({
    'time': time,
    'qsim': np.squeeze(qsim),
    'qobs': np.squeeze(qobs),
    'prcp': np.squeeze(prcp),
})
df.to_csv(r"C:\Users\20676\Desktop\songliaoyubao\dahuofang\dahuofangchangcishuju\训练期.csv", index=False)

In [2]:
#注，要全部都是英文

import netCDF4 as nc
import pandas as pd
import numpy as np

file_path = r"C:\Users\20676\Desktop\songliaoyubao\dahuofang\dahuofangchangcishuju\xaj_evaluation_results (3).nc"
data = nc.Dataset(file_path)

# 读取变量
time = nc.num2date(data.variables['time'][:], data.variables['time'].units)
qsim = data.variables['qsim'][:]
qobs = data.variables['qobs'][:]
prcp = data.variables['prcp'][:]

# 将变量转换为pandas DataFrame
df = pd.DataFrame({
    'time': time,
    'qsim': np.squeeze(qsim),
    'qobs': np.squeeze(qobs),
    'prcp': np.squeeze(prcp),
})
df.to_csv(r"C:\Users\20676\Desktop\songliaoyubao\dahuofang\dahuofangchangcishuju\验证期.csv", index=False)

合并验证期、训练期

In [2]:
import pandas as pd
# 读取csv文件
df1 = pd.read_csv(r"C:\Users\20676\Desktop\songliaoyubao\dahuofang\dahuofangchangcishuju\训练期.csv")
df2 = pd.read_csv(r"C:\Users\20676\Desktop\songliaoyubao\dahuofang\dahuofangchangcishuju\验证期.csv")

# 拼接DataFrame
df = pd.concat([df1, df2])

# 保存为新的csv文件
df.to_csv(r"C:\Users\20676\Desktop\songliaoyubao\dahuofang\dahuofangchangcishuju\新安江模拟（大伙房）.csv", index=False)

将用模型预报的数据还原为场次洪水数据

In [8]:
import pandas as pd
import os

# 读取CSV文件
df = pd.read_csv(r'C:\Users\20676\Desktop\songliaoyubao\dahuofang\dahuofangchangcishuju\新安江模拟（大伙房）.csv')

# 创建一个空的DataFrame来存储提取的数据
new_df = pd.DataFrame()

# 文件计数器
file_count = 1

for index, row in df.iterrows():
    # 检查"qobs"列的值是否为空
    if pd.isna(row['qobs']):
        # 如果值为空且new_df不为空，将当前的DataFrame保存为新的CSV文件，并清空DataFrame以便存储下一段数据
        if not new_df.empty:
            new_df.to_csv(fr'C:\Users\20676\Desktop\songliaoyubao\dahuofang\dahuofangchangcishuju\去掉预热期\data_{file_count}.csv', index=False)
            new_df = pd.DataFrame()
            file_count += 1
    else:
        # 如果值不为空，将其添加到DataFrame中
        new_df = pd.concat([new_df, pd.DataFrame(row).T])

# 保存最后一段数据
if not new_df.empty:
    new_df.to_csv(fr'C:\Users\20676\Desktop\songliaoyubao\dahuofang\dahuofangchangcishuju\去掉预热期\data_{file_count}.csv', index=False)

将松辽预报的场次洪水设置为与我们一致的格式

In [9]:
import os
import pandas as pd

# 指定源文件夹路径和目标文件夹路径
source_folder_path = r'C:\Users\20676\Desktop\songliaoyubao\水库预报结果\大伙房水库'
target_folder_path = r'C:\Users\20676\Desktop\songliaoyubao\水库预报结果\大伙房水库（重采样）'
# 如果目标文件夹不存在，创建它
if not os.path.exists(target_folder_path):
    os.makedirs(target_folder_path)

# 获取源文件夹中的所有xlsx文件
xlsx_files = [f for f in os.listdir(source_folder_path) if f.endswith('.xlsx')]

# 对于每个xlsx文件
for xlsx_file in xlsx_files:
    # 读取xlsx文件
    df = pd.read_excel(os.path.join(source_folder_path, xlsx_file))

    # 将时间列转换为datetime类型，并设置为索引
    df['时间'] = pd.to_datetime(df['时间'], format='%m-%d %H:%M')
    df.set_index('时间', inplace=True)

    # 将数据重采样为逐小时的，并填充缺失值
    df_resampled = df.resample('H').interpolate(method='linear')

    # 将数据写入新的xlsx文件
    df_resampled.to_excel(os.path.join(target_folder_path, xlsx_file))

C:\Users\20676\AppData\Local\Temp\ipykernel_20572\1086891586.py:21: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_resampled = df.resample('H').interpolate(method='linear')
C:\Users\20676\AppData\Local\Temp\ipykernel_20572\1086891586.py:21: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_resampled = df.resample('H').interpolate(method='linear')
C:\Users\20676\AppData\Local\Temp\ipykernel_20572\1086891586.py:21: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_resampled = df.resample('H').interpolate(method='linear')
C:\Users\20676\AppData\Local\Temp\ipykernel_20572\1086891586.py:21: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  d

将松辽委预报的数据转换为我们的格式添加到我们预报的后面绘图

In [13]:
import os
import pandas as pd
from datetime import datetime

# 指定源文件夹路径和目标文件夹路径
xlsx_folder_path = r'C:\Users\20676\Desktop\songliaoyubao\水库预报结果\大伙房水库（重采样）'
target_folder_path = r'C:\Users\20676\Desktop\songliaoyubao\大伙房水库(重采样）重新命名'

# 如果目标文件夹不存在，创建它
if not os.path.exists(target_folder_path):
    os.makedirs(target_folder_path)

# 获取源文件夹中的所有xlsx文件
xlsx_files = [f for f in os.listdir(xlsx_folder_path) if f.endswith('.xlsx')]

# 提取文件名中的日期部分，并将其转换为日期格式
dates = [datetime.strptime(f[-13:-5], '%Y%m%d') for f in xlsx_files]

# 根据日期对文件进行排序
sorted_files = [x for _, x in sorted(zip(dates, xlsx_files))]

# 对于每个排序后的文件
for i, xlsx_file in enumerate(sorted_files, start=1):
    # 读取xlsx文件，并将实测入流列和预报总入流列改名为qobsslw和qsimslw
    df_xlsx = pd.read_excel(os.path.join(xlsx_folder_path, xlsx_file))
    df_xlsx.rename(columns={'实测入流': 'qobsslw', '预报总入流': 'qsimslw'}, inplace=True)

    # 保存qobsslw、qsimslw列
    df_xlsx = df_xlsx[['qobsslw', 'qsimslw']]

    # 将数据写入新的csv文件，文件名为"data_i.csv"，其中i是文件的排序顺序
    df_xlsx.to_csv(os.path.join(target_folder_path, f'data_{i}.csv'), index=False)

In [14]:
import os
import pandas as pd

# 指定目标文件夹路径
target_folder_path = r'C:\Users\20676\Desktop\songliaoyubao\大伙房水库(重采样）重新命名'

# 获取目标文件夹中的所有csv文件
csv_files = [f for f in os.listdir(target_folder_path) if f.endswith('.csv')]

# 对于每个csv文件
for csv_file in csv_files:
    # 读取csv文件
    df_csv = pd.read_csv(os.path.join(target_folder_path, csv_file))

    # 删除qobsslw列为空的行
    df_csv.dropna(subset=['qobsslw'], inplace=True)

    # 将数据写回csv文件
    df_csv.to_csv(os.path.join(target_folder_path, csv_file), index=False)

将处理好的松辽预报和我们预报的数据合并

In [16]:
import os
import pandas as pd

# 指定源文件夹路径和目标文件夹路径
source_folder_path = r'C:\Users\20676\Desktop\songliaoyubao\大伙房水库(重采样）重新命名'
target_folder_path = r'C:\Users\20676\Desktop\songliaoyubao\dahuofang\dahuofangchangcishuju\去掉预热期'
final_folder_path = r'C:\Users\20676\Desktop\songliaoyubao\大伙房水库（有雨量）松辽'
# 如果目标文件夹不存在，创建它
if not os.path.exists(final_folder_path):
    os.makedirs(final_folder_path)

# 获取源文件夹中的所有csv文件
csv_files = [f for f in os.listdir(source_folder_path) if f.endswith('.csv')]

# 对于每个csv文件
for csv_file in csv_files:
    # 读取源文件夹中的csv文件，并提取qobsslw和qsimslw列
    df_source = pd.read_csv(os.path.join(source_folder_path, csv_file))[['qobsslw', 'qsimslw']]

    # 读取目标文件夹中同名的csv文件
    df_target = pd.read_csv(os.path.join(target_folder_path, csv_file))

    # 将两个DataFrame合并
    df_combined = pd.concat([df_target, df_source], axis=1)

    # 将数据写入新的csv文件
    df_combined.to_csv(os.path.join(final_folder_path, csv_file), index=False)

调用绘图函数绘制图片

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from hydroutils.hydro_plot import plot_rainfall_runoff

# 文件夹路径
folder_path = r"C:\Users\20676\Desktop\降雨径流相关图\大伙房水库（有雨量）松辽"
savey_path = r"C:\Users\20676\Desktop\降雨径流相关图\图片\大伙房"
# 获取文件夹中的所有文件
file_names = os.listdir(folder_path)

# 处理每个文件
for file_name in file_names:
    # 读取数据，调整日期格式
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    df['time'] = pd.to_datetime(df['time'], dayfirst=False)  # 修改这里
    df.set_index('time', inplace=True)
    rainfall = df['prcp']
    qsim = df['qsim']
    qobs = df['qobs']
    qsimslw = df['qsimslw']

    # 调用绘图函数
    plot_rainfall_runoff(
        t=[qsim.index, qobs.index, qsimslw.index],
        p=rainfall,
        qs=[qsim, qobs, qsimslw],
        title="Rainfall-Runoff Relationship",
        xlabel="Date",
        ylabel="Runoff (m³/s)",
        fig_size=(12, 6),
        leg_lst=["Simulated Runoff", "Observed Runoff", "Simulated Runoff with slw"]
    )

    # 保存图表
    save_path = os.path.join(savey_path, f"{file_name}.png")
    plt.savefig(save_path, dpi=300)
    plt.close()  # 关闭图表，以便开始下一个图表

下面的代码是将新安江模型预报的数据转换成xlsx格式去和松辽的计算差异

In [18]:
import os
import pandas as pd

# 指定源文件夹和目标文件夹路径
source_folder = r'C:\Users\20676\Desktop\songliaoyubao\dahuofang\dahuofangchangcishuju\去掉预热期'
target_folder = r'C:\Users\20676\Desktop\songliaoyubao\水库预报结果\大伙房水库XAJ'

# 如果目标文件夹不存在，创建它
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

# 获取源文件夹中的所有csv文件
csv_files = [f for f in os.listdir(source_folder) if f.endswith('.csv')]

# 对于每个csv文件
for csv_file in csv_files:
    # 读取csv文件
    df = pd.read_csv(os.path.join(source_folder, csv_file))

    # 将数据写入xlsx文件，保存到目标文件夹
    df.to_excel(os.path.join(target_folder, csv_file.replace('.csv', '.xlsx')), index=False)

In [21]:
import os
import pandas as pd

# 指定源文件夹路径
xlsx_folder_path = r'C:\Users\20676\Desktop\songliaoyubao\水库预报结果\大伙房水库XAJ'

# 获取源文件夹中的所有xlsx文件
xlsx_files = [f for f in os.listdir(xlsx_folder_path) if f.endswith('.xlsx')]

# 对于每个xlsx文件
for xlsx_file in xlsx_files:
    # 读取xlsx文件
    df = pd.read_excel(os.path.join(xlsx_folder_path, xlsx_file))

    # 更改列名
    df.rename(columns={'time': '时间', 'qsim': '预报总入流', 'qobs': '实测入流'}, inplace=True)

    # 只保留需要的列
    df = df[['时间', '预报总入流', '实测入流']]

    # 将修改后的DataFrame保存回xlsx文件
    df.to_excel(os.path.join(xlsx_folder_path, xlsx_file), index=False)

In [24]:
import os
import pandas as pd

# 指定源文件夹路径
xlsx_folder_path = r'C:\Users\20676\Desktop\songliaoyubao\水库预报结果\大伙房水库XAJ'

# 获取源文件夹中的所有xlsx文件
xlsx_files = [f for f in os.listdir(xlsx_folder_path) if f.endswith('.xlsx')]

# 对于每个xlsx文件
for xlsx_file in xlsx_files:
    # 读取xlsx文件
    df = pd.read_excel(os.path.join(xlsx_folder_path, xlsx_file))

    # 更改时间列的格式
    df['时间'] = pd.to_datetime(df['时间']).dt.strftime('%m-%d %H:%M')

    # 将修改后的DataFrame保存回xlsx文件
    df.to_excel(os.path.join(xlsx_folder_path, xlsx_file), index=False)

上面的代码已经将预报的大伙房水库的数据转换成我们评估的形式了，下面是两个评估的函数

import os
from datetime import datetime

import pandas as pd


def calculate_indicators(file_path):
    data = pd.read_excel(file_path, sheet_name="Sheet1")
    data_filtered = data.dropna(subset=["实测入流", "预报总入流"])

    # 检查数据是否足够计算相关系数
    if len(data_filtered) < 2:
        return {
            "correlation": None,
            "NSE": None,
            "peak_deviation_abs": None,
            "peak_deviation_percentage": None,
            "volume_deviation_abs": None,
            "volume_deviation_percentage": None,
            "peak_time_deviation": None,
        }

    # 计算相关系数
    correlation = (
        data_filtered["实测入流"].corr(data_filtered["预报总入流"])
        if len(data_filtered) > 1
        else None
    )

    # 计算NSE，防止分母为0
    observed = data_filtered["实测入流"]
    predicted = data_filtered["预报总入流"]
    denominator = ((observed - observed.mean()) ** 2).sum()
    nse = (
        1 - ((predicted - observed) ** 2).sum() / denominator
        if denominator != 0
        else None
    )

    # 计算洪峰偏差
    peak_measured = observed.max() if not observed.empty else None
    peak_predicted = predicted.max() if not predicted.empty else None
    if peak_measured is not None and peak_predicted is not None:
        peak_deviation_abs = abs(peak_measured - peak_predicted)
        peak_deviation_percentage = (peak_deviation_abs / peak_measured) * 100
    else:
        peak_deviation_abs = None
        peak_deviation_percentage = None

    # 计算洪量偏差
    total_volume_measured = observed.sum()
    total_volume_predicted = predicted.sum()
    if total_volume_measured != 0:
        volume_deviation_abs = abs(total_volume_measured - total_volume_predicted)
        volume_deviation_percentage = (
            volume_deviation_abs / total_volume_measured
        ) * 100
    else:
        volume_deviation_abs = None
        volume_deviation_percentage = None

    # 计算峰现时间偏差
    if not data_filtered.empty and "时间" in data_filtered.columns:
        try:
            time_of_peak_measured = data_filtered.loc[
                data_filtered["实测入流"] == peak_measured, "时间"
            ].iloc[0]
            time_of_peak_predicted = data_filtered.loc[
                data_filtered["预报总入流"] == peak_predicted, "时间"
            ].iloc[0]
            time_of_peak_measured = pd.to_datetime(
                time_of_peak_measured, format="%m-%d %H:%M"
            )
            time_of_peak_predicted = pd.to_datetime(
                time_of_peak_predicted, format="%m-%d %H:%M"
            )
            peak_time_deviation = abs(
                (time_of_peak_measured - time_of_peak_predicted).total_seconds() / 3600
            )
        except IndexError:
            peak_time_deviation = None
    else:
        peak_time_deviation = None

    return {
        "correlation": correlation,
        "NSE": nse,
        "peak_deviation_abs": peak_deviation_abs,
        "peak_deviation_percentage": peak_deviation_percentage,
        "volume_deviation_abs": volume_deviation_abs,
        "volume_deviation_percentage": volume_deviation_percentage,
        "peak_time_deviation": peak_time_deviation,
    }

In [ ]:
# 在处理DataFrame和检查合格率时，确保在尝试计算之前列已经存在
def process_folder_with_stats(folder_path):
    results = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".xlsx"):
            file_path = os.path.join(folder_path, filename)
            indicators = calculate_indicators(file_path)
            if indicators:  # 确保指标非空
                results.append(indicators)

    # 创建DataFrame
    if not results:
        return {}
    results_df = pd.DataFrame(results)

    # 检查必要的列是否存在
    if (
        "peak_deviation_percentage" in results_df.columns
        and "volume_deviation_percentage" in results_df.columns
    ):
        results_df["is_peak_qualified"] = results_df["peak_deviation_percentage"] <= 20
        results_df["is_volume_qualified"] = (
            results_df["volume_deviation_percentage"] <= 20
        )
        results_df["is_qualified"] = (
            results_df["is_peak_qualified"] & results_df["is_volume_qualified"]
        )
        qualification_rate = results_df["is_qualified"].mean()
    else:
        qualification_rate = None

    # 计算NSE和相关系数的统计值
    nse_median = results_df["NSE"].median() if "NSE" in results_df.columns else None
    nse_mean = results_df["NSE"].mean() if "NSE" in results_df.columns else None
    corr_median = (
        results_df["correlation"].median()
        if "correlation" in results_df.columns
        else None
    )
    corr_mean = (
        results_df["correlation"].mean()
        if "correlation" in results_df.columns
        else None
    )

    return {
        "Qualification Rate": qualification_rate,
        "NSE Median": nse_median,
        "NSE Mean": nse_mean,
        "Correlation Median": corr_median,
        "Correlation Mean": corr_mean,
    }

调用函数评估

In [ ]:
folder_path = "水库预报结果/大伙房水库XAJ"
stats = process_folder_with_stats(folder_path)
stats

调用函数评估将松辽的预报转换为逐小时的

In [ ]:
folder_path = "水库预报结果/大伙房水库（重采样）"
stats = process_folder_with_stats(folder_path)
stats